## **Random-Classifier**

**Import necessary packages**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import yaml
import numpy as np

**Load the YAML file and construct full feature names**

In [ ]:
with open('/content/drive/MyDrive/FYP/Old codes/Feature Extraction/params.yaml', 'r') as file:
    params_data = yaml.safe_load(file)

**Construct feature column names based on the hierarchical structure in params.yaml**

In [ ]:
feature_columns = [f"original_{feature_class}_{feature}" for feature_class, features in params_data['featureClass'].items() for feature in features]

**Load the data**

In [ ]:
normal_df = pd.read_csv('/content/drive/MyDrive/FYP/extracted_features_normal.csv')
covid_df = pd.read_csv('/content/drive/MyDrive/FYP/extracted_features_Covid.csv')

**Assign labels and combine datasets**

In [ ]:
normal_df['Target'] = 0  # Label for 'Normal' cases
covid_df['Target'] = 1   # Label for 'COVID' cases
df = pd.concat([normal_df, covid_df], ignore_index=True)

**Check for missing feature columns**

In [ ]:
missing_features = [col for col in feature_columns if col not in df.columns]
if missing_features:
    print("The following features are missing from the dataset:", missing_features)

**Select only the feature columns and the target**

In [ ]:
X = df[feature_columns]
y = df['Target']

**Data Preprocessing**

In [ ]:
X.fillna(X.mean(), inplace=True)  # Fill missing values with column mean
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**Train-test split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

**Initialize and train Random Forest classifier**

In [ ]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

**Cross-validation**

In [ ]:
cv_scores = cross_val_score(rf_clf, X_scaled, y, cv=5)
print("Cross-validation accuracy scores:", cv_scores)
print("Mean cross-validation accuracy:", cv_scores.mean())

**Predictions and evaluation**

In [ ]:
y_pred = rf_clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

**Confusion Matrix**

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf_clf.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

**Feature Importance**

In [ ]:
feature_importances = rf_clf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': feature_columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df.head(10), palette='viridis')
plt.title("Top 10 Feature Importances")
plt.show()

**Heatmap of Feature Correlations**

In [ ]:
corr_matrix = pd.DataFrame(X_scaled, columns=feature_columns).corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False, fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

**Cross-Validation Scores Plot**

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, 6), cv_scores, marker='o', linestyle='-', color='b')
plt.title("Cross-Validation Scores across Folds")
plt.xlabel("Fold")
plt.ylabel("Accuracy Score")
plt.ylim(0, 1)
plt.grid()
plt.show()